In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_train.head()

In [ ]:
df_test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
df_test.head()

In [ ]:
df_train.shape, df_test.shape

In [ ]:
#combine train and test data for data preprocessing
df_merge=pd.concat([df_test.assign(ind="test"), df_train.assign(ind="train")])
df_merge.head()

In [ ]:
#Get count of missing values in each column
def get_cols_with_missing_values(DataFrame):
    missing_na_columns=(DataFrame.isnull().sum())
    return missing_na_columns[missing_na_columns > 0]

print(get_cols_with_missing_values(df_merge))

In [ ]:
df_merge.info()

In [ ]:
df_merge.drop('Id',axis=1,inplace=True)

In [ ]:
categorical_cols = [cname for cname in df_merge.columns if
                    df_merge[cname].nunique() < 10 and 
                    df_merge[cname].dtype == "object"]
print(categorical_cols)

In [ ]:
missing_cat_cols=['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']

df_merge[missing_cat_cols]=df_merge[missing_cat_cols].fillna("NO")

In [ ]:
df_merge_clean = df_merge.loc[:, df_merge.columns!='SalePrice'].apply(lambda x:x.fillna(x.value_counts().index[0]))

In [ ]:
df_merge_clean2 = pd.concat([df_merge_clean, df_merge['SalePrice']], axis=1)

In [ ]:
#Ignore SalePrice as it is Target feature
print(get_cols_with_missing_values(df_merge_clean2))

In [ ]:
df_merge_clean2.head()

In [ ]:
categorical_cols = [cname for cname in df_merge_clean2.columns if
                    df_merge_clean2[cname].nunique() < 10 and 
                    df_merge_clean2[cname].dtype == "object"]

numerical_cols=[cname for cname in df_merge_clean2.columns if df_merge_clean2[cname].dtypes!='object']

my_cols = numerical_cols + categorical_cols


In [ ]:
df_merge_clean3 = df_merge_clean2[my_cols].copy()
df_merge_clean3.shape

In [ ]:
df_merge_clean3['Street'].unique()

Street: Type of road access to property

       Grvl	Gravel	
       Pave	Paved

In [ ]:
df_merge_clean3['Street']=df_merge_clean3['Street'].map({"Pave":1, "Grvl":2}).astype(int)

In [ ]:
df_merge_clean3['Alley'].unique()

Alley: Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access

In [ ]:
df_merge_clean3['Alley']=df_merge_clean3['Alley'].map({"NO":0, "Pave":1, "Grvl":2}).astype(int)

In [ ]:
df_merge_clean3['LotShape'].unique()

LotShape: General shape of property

       Reg	Regular	
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular


In [ ]:
df_merge_clean3['LotShape']=df_merge_clean3['LotShape'].map({"IR3":0, "IR2":1, "IR1":2, "Reg":3}).astype(int)

In [ ]:
df_merge_clean3['LandContour'].unique()

LandContour: Flatness of the property

       Lvl	Near Flat/Level	
       Bnk	Banked - Quick and significant rise from street grade to building
       HLS	Hillside - Significant slope from side to side
       Low	Depression

In [ ]:
df_merge_clean3['LandContour']=df_merge_clean3['LandContour'].map({"Low":0, "HLS":1, "Bnk":2, "Lvl":3}).astype(int)

In [ ]:
df_merge_clean3['Utilities'].unique()

Utilities: Type of utilities available
		
       AllPub	All public Utilities (E,G,W,& S)	
       NoSewr	Electricity, Gas, and Water (Septic Tank)
       NoSeWa	Electricity and Gas Only
       ELO	Electricity only	

In [ ]:
df_merge_clean3['Utilities']=df_merge_clean3['Utilities'].map({"AllPub":0, "NoSeWa":1}).astype(int)

In [ ]:
df_merge_clean3['LotConfig'].unique()

In [ ]:
#split test and train data
test, train= df_merge_clean2[df_merge_clean2["ind"].eq("test")], df_merge_clean2[df_merge_clean2["ind"].eq("train")]
test.drop(["SalePrice", "ind"], axis=1, inplace=True)
train.drop(["ind"], axis=1, inplace=True)